# Test of the Triplex Extractor object

### Import ontoconnectlm package

In [ ]:
import sys

sys.path.append("../")

This notebook describes the application of a Triplet Extractor to an input text.
- Objective: Extract subject-predicate-object triplets from text using a specified LLM model.
- Steps:
    - Specify the input text: Define the text to be processed for triplet extraction.
     -   Configure LLM Model Parameters: 
        - base_url: Specify the endpoint URL of the LLM model. 
        - model: Provide the model's name or identifier. 
    - Define Contextual Information: context_description, entity_descriptions and relation_types

### Test import

In [ ]:
from ontoconnectlm.triplet_extractor import TripletExtractor

### Input data (end-user)

In [ ]:
# Define the LLM object you would like to use
from langchain_ollama import OllamaLLM

llm = OllamaLLM(base_url="127.0.0.1:11434" , model="mistral-small3.1:24b")

context_description = "Les textes portent sur des incidents sur un réseau électrique."

entity_types = [
    "Event",
    "Datetime",
    "Client",
    "Location",
    "PowerComponent"
]

entity_descriptions = {
    "Event" : "Nom d'un événement pouvant survenir sur le réseau électrique.",
    "Datetime" : "Indique la temporalité d'une action, d'un événement. Il s'agit d'une heure mais pas d'une date. Est souvent indiquée en début de paragraphe.",
    "Client" : "Nom d'un client ou partenaire du gestionnaire du réseau.",
    "Location" : "La zone géographique constitue tout lieu qui ne représente pas une infrastructure ou une division du gestionnaire électrique. Il s'agit le plus souvent de villes, de régions, de pays.",
    "PowerComponent" : "Fait référence à un élément d'infrastructure du gestionnaire de réseau électrique. Il peut s'agir de différents types d'objets : transformateurs, lignes, postes."
}

relation_types = [
    "occuredAtTime",
    "occuredAtPlace",
    "hasOrigin",
    "affectedComponent",
    "impactedClient",
]

relation_descriptions = {
    "occuredAtTime" : "Event --> Datetime",
    "occuredAtPlace" :  "Event --> Location",
    "hasOrigin" :  "Event --> Event",
    "affectedComponent" :  "Event --> PowerComponent",
    "impactedClient" :  "Event --> Client"
}

extractor = TripletExtractor(
    llm = llm,
    context_description=context_description,
    entity_types=entity_types,
    entity_descriptions = entity_descriptions,
    relation_types=relation_types,
    relation_descriptions = relation_descriptions
)

# Mode Open Relation Extraction

# extractor = TripletExtractor(
#     llm = llm,
#     context_description=context_description,
# )

### Données de test

In [ ]:
# Données réalistes avec valeurs modifiées

texts = [
    """Date et heure: 20/04/2012 à 07h11
Frontière: Belgique
Impact(s) et action(s) réalisée(s): Détection d'oscillation de fréquence à la frontière France Belgique par IDE. Pas d'évènement pouvant expliquer ces oscillations en France.""",

"""Date et heure: 08/07/2003 à 08h40
Centre: Haute-Normandie
Ouvrage(s) concerné(s): Coupure longue de l'ACR de Nogent-sur-Marne au poste de Villejuif 225 kV (16 MW pendant 5 minutes) suite au déclenchement de la liaison 225 kV MASSY-PALAISEAU 4. Cf. MIN.
Impact(s): Cf. MIN.""",

"""Date et heure: 19/04/2016 à 16h32
Centre: Bordeaux
Ouvrage(s) concerné(s): Coupure longue des clients SystemX au poste de GIF 63 kV et CEA au poste de ETABLI 93 kV suite au déclenchement des lignes 93 kV TONNERRE-NUITS-SAINT-GEORGES 1 et BORDEAUX-SAINT-JEAN 93 kV.
Impact(s): Coupure du CNRS."""
]

### Test instanciation

In [ ]:
from langchain_ollama import OllamaLLM

# Set LLM configuration parameters
llm = OllamaLLM(base_url="127.0.0.1:11434" , model="mistral-small3.1:24b")

# Define context associated to the input data
context_description = "Les textes portent sur des incidents sur un réseau électrique."

# List types of entities to be extracted from the input text 
entity_types = [
    "Event",
    "Datetime",
    "Client",
    "Location",
    "PowerComponent"
]

# Provide associated decsriptions to entites to be extracted 
entity_descriptions = {
    "Event" : "Nom d'un événement pouvant survenir sur le réseau électrique.",
    "Datetime" : "Indique la temporalité d'une action, d'un événement. Il s'agit d'une heure mais pas d'une date. Est souvent indiquée en début de paragraphe.",
    "Client" : "Nom d'un client ou partenaire du gestionnaire du réseau.",
    "Location" : "La zone géographique constitue tout lieu qui ne représente pas une infrastructure ou une division du gestionnaire électrique. Il s'agit le plus souvent de villes, de régions, de pays.",
    "PowerComponent" : "Fait référence à un élément d'infrastructure du gestionnaire de réseau électrique. Il peut s'agir de différents types d'objets : transformateurs, lignes, postes."
}

# List relation types to be extracted from the input text and used 
relation_types = [
    "occuredAtTime",
    "occuredAtPlace",
    "hasOrigin",
    "affectedComponent",
    "impactedClient",
]

# Indicate the domain and the range associated to each target relation
relation_descriptions = {
    "occuredAtTime" : "Event --> Datetime",
    "occuredAtPlace" :  "Event --> Location",
    "hasOrigin" :  "Event --> Event",
    "affectedComponent" :  "Event --> PowerComponent",
    "impactedClient" :  "Event --> Client"
}

# Setthe llm model parameters
extractor = TripletExtractor(
    llm = llm,
    context_description=context_description,
    entity_types=entity_types,
    entity_descriptions = entity_descriptions,
    relation_types=relation_types,
    relation_descriptions = relation_descriptions
)

### Test triples generation

In [ ]:
# Predict list of triples from the input text
results = extractor.run(texts) 

In [ ]:
results

### Display results

In [ ]:
# List all the predicted triples related to the input text
for text_triples in results:
    for triplet in text_triples:
        print(triplet["head"] , " (" , triplet["head_type"] , ") " , triplet["label"], triplet["tail"] , " (" , triplet["tail_type"] , ") ")